
In the Google shared drive (/602/data), the file enron.txt is a subset of the Enron Corpus, a collection of over 500,000 emails from senior management of Enron Corporation leading to its collapse in 2001.  The subset comprises the text of about 15,000 emails available through the TensorFlow Data Set (TFDS) source aeslc (annotated Enron Subject Line Corpus).

Using this dataset, construct a neural net that will generate 50 random characters, beginning with the sequence \verb!The!, that are generated from the distribution of text in the file.

This exercise can be replicated using any of the following sources in the texts and documentation:

* **Raschka** - Character-level language modeling in TensorFlow, pages 600-613
* **Gèron** - Generating Shakespearean Text Using a Character RNN, pages 526-534
* **TensorFlow documentation** [Text Generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation)


Adjust the temperature ($\alpha$ in Raschka) to avoid repeating text.  Using a GPU runtime to fit the model is advised, which may still require several hours to train.


In [1]:
from google.colab import drive
import numpy as np
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#read the file content into the variable corpus
with open('/content/drive/Shareddrives/DS602-S22/Data/enron.txt', 'r', encoding='utf8') as f:
  corpus = f.read()

In [3]:
print(corpus[0:1000])

Greg and Mark:  Attached is a draft of the very short story that will accompany your profiles in Enron Business.
(PR management has approved.)
The purpose is simply to introduce you and quickly address the issue that's on everyone's mind, the stock price.
Please review your individual quotes and let me know if you have any revisions as soon as possible.
The quotes were extracted from your interviews with us a few weeks ago.
If I have not heard back from you by the end of the day tomorrow, I'll assume it's OK to print as is.
Thanks again for taking the time to meet with us.
Warm regards and cheers,
Please note the change in location for tomorrow's meeting.
EB4102 is undergoing construction for several months.
Ops Committee Meeting 		Friday, 10/5 		1:30 - 2:30 PM 		EB30C2 - OMA 3C1  Thanks.
John/John/Frank  As we are approaching the November 1st Trading Track interview dates, we need to finalize the reminder of the external candidate's initial screen.
Can you please provide me with your 

In [4]:
import tensorflow as tf
import numpy as np
import os
import time

In [5]:
enron_data='/content/drive/Shareddrives/DS602-S22/Data/enron.txt'

In [6]:
text = open(enron_data, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 14442507 characters


In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

97 unique characters


In [8]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
text_ids=ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))
text_ids

<tf.Tensor: shape=(14442507,), dtype=int64, numpy=array([43, 85, 72, ..., 86,  3,  2])>

In [11]:
input = tf.data.Dataset.from_tensor_slices(text_ids)
input

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [12]:
for i in input.take(15):
    print(chars_from_ids(i).numpy().decode('utf-8'),end=" ")

G r e g   a n d   M a r k :   

In [13]:
seq_length = 80
ex_per_epoch = len(text)//(seq_length+1)
print(ex_per_epoch)

178302


In [14]:
sequences = input.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'G' b'r' b'e' b'g' b' ' b'a' b'n' b'd' b' ' b'M' b'a' b'r' b'k' b':'
 b' ' b' ' b'A' b't' b't' b'a' b'c' b'h' b'e' b'd' b' ' b'i' b's' b' '
 b'a' b' ' b'd' b'r' b'a' b'f' b't' b' ' b'o' b'f' b' ' b't' b'h' b'e'
 b' ' b'v' b'e' b'r' b'y' b' ' b's' b'h' b'o' b'r' b't' b' ' b's' b't'
 b'o' b'r' b'y' b' ' b't' b'h' b'a' b't' b' ' b'w' b'i' b'l' b'l' b' '
 b'a' b'c' b'c' b'o' b'm' b'p' b'a' b'n' b'y' b' ' b'y'], shape=(81,), dtype=string)


In [15]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [16]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Greg and Mark:  Attached is a draft of the very short story that will accompany y'
b'our profiles in Enron Business.\r\n(PR management has approved.)\r\nThe purpose is si'
b"mply to introduce you and quickly address the issue that's on everyone's mind, th"
b'e stock price.\r\nPlease review your individual quotes and let me know if you have '
b'any revisions as soon as possible.\r\nThe quotes were extracted from your interview'


In [17]:
def target_split(seq):
  return seq[:-1], seq[1:]

In [18]:
dataset = sequences.map(target_split)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Greg and Mark:  Attached is a draft of the very short story that will accompany '
Target: b'reg and Mark:  Attached is a draft of the very short story that will accompany y'


In [20]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 80), dtype=tf.int64, name=None), TensorSpec(shape=(64, 80), dtype=tf.int64, name=None))>

In [21]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 80, 98) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  25088     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  100450    
                                                                 
Total params: 4,063,842
Trainable params: 4,063,842
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([32, 83, 91, 25, 75, 12, 42, 73, 22, 94, 64, 47, 33, 51, 55, 81, 32,
       84, 67, 96, 27, 38, 21, 36, 32, 35, 76, 43, 31, 53, 82, 84, 22, 94,
       12, 40, 85, 37, 33,  5, 66, 95, 74, 37, 39, 33, 41, 90, 11, 53, 24,
        9, 54, 84, 78, 14, 71, 25, 96, 12, 35, 89, 65, 27,  1, 97, 21, 23,
       66, 25,  2, 19, 59, 13, 21, 51, 34, 65, 15, 16])

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'urate documents are distributed, please allow us to prepare the drafts for distr'

Next Char Predictions:
 b"<px5h(Ff2{\\K=OSn<q`}7B1@<?iG;Qoq2{(DrA=!_|gAC=Ew'Q4%Rqk*d5}(?v]7\t~13_5\n/W)1O>]+,"


In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
2785/2785 [==============================] - 316s 111ms/step - loss: 1.4372
Epoch 2/10
2785/2785 [==============================] - 307s 110ms/step - loss: 1.1018
Epoch 3/10
2785/2785 [==============================] - 309s 110ms/step - loss: 1.0447
Epoch 4/10
2785/2785 [==============================] - 308s 110ms/step - loss: 1.0153
Epoch 5/10
2785/2785 [==============================] - 311s 111ms/step - loss: 0.9990
Epoch 6/10
2785/2785 [==============================] - 310s 111ms/step - loss: 0.9898
Epoch 7/10
2785/2785 [==============================] - 310s 111ms/step - loss: 0.9853
Epoch 8/10
2785/2785 [==============================] - 311s 111ms/step - loss: 0.9841
Epoch 9/10
2785/2785 [==============================] - 310s 111ms/step - loss: 0.9857
Epoch 10/10
2785/2785 [==============================] - 310s 111ms/step - loss: 0.9897


In [33]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [34]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [49]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [54]:
start = time.time()
states = None
next_char = tf.constant(['The ', 'The ', 'The ', 'The ', 'The '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'The explosion "allow yet along then bagance of doesn\'t".\r\nNext meeating, Enron would have to specify our best in the world, but I think the caller we want to build cheat from Sengency in sutcome certain that this product is still for nothing, How added that there is now a things next Fred and she would give you my questions.\r\nWe have last specific on the lunch with  more late time for this.\r\nLet me prip a chance to look atmeal --3 and Marc,  team.\r\nI think it\'s now.\r\nTo do a Termination EreatR  9/18/00)     TETCO WARS TO GET THROUGY AT BC ACTELS.ONT TA TALE ME TO ENPLAYERES METER IN THE ENTERSATION INTURATED WIAL FRODBW. 11:00 AM CHECIFIMED THAN AOLARD SHATE) will occas TW matter agreements to the IRSC estate.\r\nThe subject of this master netwing the  obligation contained in the current funuse to  you up.\r\nAs you should now hear, but I want  to find this to let you know what projects from Enron While are you again and what books from hoist of this year at E

In [55]:
states = None
next_char = tf.constant(['The '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

The  language should print put their last spreads between Credit Watch Hills Bricing Dierest Name a.
- 


## Reference:

### 1. https://www.tensorflow.org/text/tutorials/text_generation